In [1]:
import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=False)

W0806 15:39:12.770999 140144563164928 deprecation.py:323] From <ipython-input-1-a59898dfbfea>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 15:39:12.771908 140144563164928 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 15:39:12.772472 140144563164928 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz


W0806 15:39:12.992454 140144563164928 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
# 입력값과 출력값을 받기 위한 플레이스홀데를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.int64, shape=[None])

In [3]:
# 변수들을 설정하고 소프트맥스 회귀 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784,10]), name='W')
b = tf.Variable(tf.zeros(shape=[10]), name='b')
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

In [4]:
# cross-engropy 손실 함수와 옵티마이저를 정의합니다.
#loss_function = -tf.reduce_sum(y*tf.log(y_pred), reduction_indices=[1])
loss_function = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss_function)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 텐서보드를 위한 요약 정보(scalar)를 정의합니다.
tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss:0' shape=() dtype=string>

In [5]:
# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 텐서보드 요약정보들을 하나로 합칩니다.
merged = tf.summary.merge_all()
# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

In [6]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys})
    summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys})
    tensorboard_writer.add_summary(summary, i)
    if i%1000==999:
        print("#",i+1,": ", sess.run(loss_function, feed_dict={x:batch_xs, y:batch_ys}))

# 1000 :  [8.50460410e-01 1.13687320e-02 6.75575286e-02 4.91732173e-02
 3.92310182e-03 2.03153864e-01 1.88521773e-01 2.02652678e-01
 4.06479299e-01 1.40707278e+00 2.14576494e-06 8.30068626e-03
 2.83592492e-02 9.85970604e-04 9.28838491e-01 1.02872327e-04
 1.45582808e-02 8.28057766e-01 1.51321553e-02 7.78202200e-03
 2.26033740e-02 2.81559825e-02 9.84659910e-01 6.02596641e-01
 1.36962088e-04 3.40545326e-02 5.45948686e-04 1.99470520e-02
 2.02382114e-02 1.69361889e-01 5.50243584e-03 6.73343897e-01
 8.21170360e-02 5.89236498e-01 6.68953499e-03 1.21265508e-01
 9.18626413e-03 2.79730763e-02 3.27063799e-02 1.21210038e-03
 1.85426380e-02 4.95898348e-05 2.87934840e-01 4.24575806e-02
 1.49145350e-01 3.13755065e-01 1.23976897e-05 3.49470749e-02
 6.15065694e-01 9.92880985e-02 9.29209311e-03 1.11418664e-01
 3.47204655e-01 1.77488080e-03 3.57627943e-02 1.35897662e-05
 3.77455205e-02 3.74843925e-01 1.48298731e-02 3.04288477e-01
 6.96059142e-04 2.45103613e-03 9.98761784e-03 8.36107805e-02
 6.51920557e-0

In [7]:
# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('정확도: %f' %sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))

정확도: 0.922000


In [8]:
print('W=', sess.run(W))
print('b=', sess.run(b))

W= [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
b= [-1.007275    0.9050872   0.19287491 -0.57942736  0.02008091  2.3380344
 -0.40095255  1.3420058  -2.3460639  -0.46434224]


In [9]:
sess.close()